### 0. Ścieżki

In [2]:


### Ścieżki wejścia/wyjścia dla modułu "Quality Metrics"

from pathlib import Path

PROJECT_ROOT = Path(".").resolve()

# katalog z obrazami
IMAGE_ROOT = PROJECT_ROOT / "inputs"   # jeżeli masz inną ścieżkę, zostaw swoją
print("IMAGE_ROOT:", IMAGE_ROOT, "| istnieje:", IMAGE_ROOT.exists())

# katalog artefaktów jakości (opcjonalny: miniatury, wykresy)
OUTPUT_DIR = PROJECT_ROOT / "outputs" /"csv"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# wspólny katalog na CSV całego pipeline'u
OUTPUT_CSV_DIR = PROJECT_ROOT / "outputs" / "csv"
OUTPUT_CSV_DIR.mkdir(parents=True, exist_ok=True)

# finalny CSV z metrykami jakości
QUALITY_CSV = OUTPUT_CSV_DIR / "quality_metrics.csv"

print("QUALITY_CSV:", QUALITY_CSV)

IMAGE_ROOT: /Users/olga/MetaLogic/inputs | istnieje: True
QUALITY_CSV: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 01 – Importy + ścieżki + lista obrazów

In [3]:
# %%
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image

DIR_DATA = Path("inputs")

def list_images(dir_path: Path):
    exts = {".jpg", ".jpeg", ".png", ".tif", ".tiff"}
    return sorted([p for p in dir_path.iterdir() if p.suffix.lower() in exts])

image_paths = list_images(DIR_DATA)
print("Liczba obrazów:", len(image_paths))

Liczba obrazów: 74


### 02 – Metryki jakości

In [4]:
# %%
def load_cv2_gray(path: Path):
    """
    Wczytuje obraz w skali szarości przez OpenCV.

    Zwraca:
    - tablicę uint8 (H, W) jeśli się udało,
    - None, jeśli OpenCV nie potrafi wczytać pliku (np. nietypowy TIFF/WEBP).
    """
    try:
        img_gray = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(f"[QUALITY] Błąd odczytu pliku {path.name}: {e}")
        return None

    if img_gray is None:
        print(f"[QUALITY] OpenCV nie wczytał pliku (None): {path.name}")
        return None

    return img_gray


def blur_score(img_gray):
    return float(cv2.Laplacian(img_gray, cv2.CV_64F).var())


def contrast_score(img_gray):
    return float(img_gray.std())


def brightness_score(img_gray):
    return float(img_gray.mean())


def compute_quality(path: Path):
    """
    Liczy metryki jakości dla pojedynczego obrazu.

    Zwraca dict:
    - file_path, blur, contrast, brightness
    lub None, jeśli obraz nie został wczytany.
    """
    img_gray = load_cv2_gray(path)
    if img_gray is None:
        return None

    return {
        "file_path": str(path),
        "blur": blur_score(img_gray),
        "contrast": contrast_score(img_gray),
        "brightness": brightness_score(img_gray),
    }

### 03 – Batch

In [5]:
# %%
records = []

for i, p in enumerate(image_paths, start=1):
    print(f"[{i}/{len(image_paths)}] {p.name}")
    rec = compute_quality(p)
    if rec:
        records.append(rec)

df_quality = pd.DataFrame(records)
df_quality



[1/74] 0004.jpg
[2/74] 0006.jpg
[3/74] 0009.jpg
[4/74] 0022.jpg
[5/74] 0034.jpg
[6/74] 0043.jpg
[7/74] 0044.jpg
[8/74] 0074.jpg
[9/74] 0075.jpg
[10/74] 0077.jpg
[11/74] 0106.jpg
[12/74] 0109.jpg
[13/74] 0111.jpg
[14/74] 0112.jpg
[15/74] 0114.jpg
[16/74] 0115 2.jpg
[17/74] 0115.jpg
[18/74] 0116.jpg
[19/74] 0117.jpg
[20/74] 0119.jpg
[21/74] 0120.jpg
[22/74] 0121.jpg
[23/74] 0124 2.jpg
[24/74] 0124.jpg
[25/74] 0125.jpg
[26/74] 0126.jpg
[27/74] 0127.jpg
[28/74] 0128.jpg
[29/74] 0135.jpg
[30/74] 0136.jpg
[31/74] 0138.jpg
[32/74] 0143.jpg
[33/74] 0145.jpg
[34/74] 0161.jpg
[35/74] 0165.jpg
[36/74] 0171.jpg
[37/74] 0174.jpg
[38/74] 0175.jpg
[39/74] 0187 2.jpg
[40/74] 0187.jpg
[41/74] 0188.jpg
[42/74] 0189.jpg
[43/74] 0196.jpg
[44/74] 0203.jpg
[45/74] 0204.jpg
[46/74] 0205.jpg
[47/74] 0206.jpg
[48/74] 0211.jpg
[49/74] 0223.jpg
[50/74] 0226.jpg
[51/74] 0229.jpg
[52/74] 0233.jpg
[53/74] 0241.jpg
[54/74] 0242.jpg
[55/74] 0253.jpg
[56/74] 0256.jpg
[57/74] 0275.jpg
[58/74] 0282.jpg
[59/74] 0286.jpg


,file_path,blur,contrast,brightness
0,inputs/0004.jpg,217.266330,84.594967,94.090657
1,inputs/0006.jpg,1292.408395,61.694697,136.716768
2,inputs/0009.jpg,1598.740746,70.362099,119.924682
3,inputs/0022.jpg,1313.585982,60.515637,112.491175
4,inputs/0034.jpg,769.719032,69.227966,146.110244
...,...,...,...,...
69,inputs/default.jpg,183.868837,68.126906,124.180634
70,inputs/djjefault-6.jpg,207.015941,52.763417,114.342822
71,inputs/fg.jpg,368.405046,65.225553,117.281366
72,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073


### CSV

In [6]:
# %%
out_path = Path("outputs/quality_metrics.csv")
df_quality.to_csv(QUALITY_CSV, index=False)
print("Zapisano metryki jakości do:", QUALITY_CSV)

Zapisano metryki jakości do: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv


### 1. Sortowanie zdjęć po ostrości (blur)

In [7]:
# %%
"""
Sortowanie zdjęć po ostrości (blur).

Tworzy dwie tabele:
- df_sharpest  – zdjęcia od najostrzejszych
- df_blurriest – zdjęcia od najbardziej rozmytych
"""

df_sharpest = df_quality.sort_values("blur", ascending=False).reset_index(drop=True)
df_blurriest = df_quality.sort_values("blur", ascending=True).reset_index(drop=True)

print("Najostrzejsze 10 zdjęć:")
df_sharpest.head(10)

Najostrzejsze 10 zdjęć:


,file_path,blur,contrast,brightness
0,inputs/0117.jpg,3991.757403,84.260545,141.577361
1,inputs/0119.jpg,3535.061055,55.277084,126.753168
2,inputs/0233.jpg,2040.941630,59.834598,119.076823
3,inputs/0242.jpg,1805.842030,56.525427,115.524981
4,inputs/0241.jpg,1794.293993,49.054932,113.971275
5,inputs/0121.jpg,1733.936451,72.261555,112.207409
6,inputs/0009.jpg,1598.740746,70.362099,119.924682
7,inputs/0077.jpg,1401.519361,57.138214,102.661564
8,inputs/0022.jpg,1313.585982,60.515637,112.491175
9,inputs/0006.jpg,1292.408395,61.694697,136.716768


In [8]:
# %%
print("Najbardziej rozmyte 10 zdjęć:")
df_blurriest.head(10)

Najbardziej rozmyte 10 zdjęć:


,file_path,blur,contrast,brightness
0,inputs/0226.jpg,41.226857,95.203951,172.202915
1,inputs/0188.jpg,74.551545,57.136059,180.875610
2,inputs/0136.jpg,77.113161,61.731172,135.309397
3,inputs/default-4.jpg,86.640338,83.075988,80.469344
4,inputs/0125.jpg,105.324313,58.087298,173.330123
5,inputs/0074.jpg,107.961643,64.880026,116.425343
6,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073
7,inputs/0161.jpg,110.241569,60.878537,157.209427
8,inputs/default-5.jpg,112.878092,86.334072,119.070590
9,inputs/0124.jpg,115.316205,66.185200,131.595816


### 2. Filtrowanie „najgorszych” jakościowo (np. 10% najbardziej rozmytych)

In [9]:
# %%
"""
Wybór najgorszych jakościowo zdjęć pod względem rozmycia.

Wyznacza próg jako 10. percentyl wartości 'blur'
i zwraca wszystkie zdjęcia poniżej tego progu.
"""

blur_threshold = df_quality["blur"].quantile(0.10)
print("Próg blur (10% najbardziej rozmyte):", blur_threshold)

df_worst_blur = df_quality[df_quality["blur"] <= blur_threshold].copy()
df_worst_blur = df_worst_blur.sort_values("blur", ascending=True).reset_index(drop=True)

print("Liczba zdjęć uznanych za najbardziej rozmyte:", len(df_worst_blur))
df_worst_blur

Próg blur (10% najbardziej rozmyte): 111.03252606582004
Liczba zdjęć uznanych za najbardziej rozmyte: 8


,file_path,blur,contrast,brightness
0,inputs/0226.jpg,41.226857,95.203951,172.202915
1,inputs/0188.jpg,74.551545,57.136059,180.875610
2,inputs/0136.jpg,77.113161,61.731172,135.309397
3,inputs/default-4.jpg,86.640338,83.075988,80.469344
4,inputs/0125.jpg,105.324313,58.087298,173.330123
5,inputs/0074.jpg,107.961643,64.880026,116.425343
6,inputs/hhdefault-4.jpg,109.300670,63.062448,118.613073
7,inputs/0161.jpg,110.241569,60.878537,157.209427


In [10]:
### X. Progi i flagi jakości

"""
Dodanie kolumn flag jakościowych na podstawie prostych progów.

df_quality      – DataFrame z metrykami jakości (blur / brightness / contrast)
BLUR_MIN_GOOD   – minimalna wartość blur uznawana za „wystarczającą ostrość”
BRIGHTNESS_MIN  – dolna granica akceptowalnej jasności
BRIGHTNESS_MAX  – górna granica akceptowalnej jasności
CONTRAST_MIN    – dolna granica akceptowalnego kontrastu
CONTRAST_MAX    – górna granica akceptowalnego kontrastu
"""

import pandas as pd

BLUR_MIN_GOOD = 50.0
BRIGHTNESS_MIN = 0.25
BRIGHTNESS_MAX = 0.85
CONTRAST_MIN = 0.15
CONTRAST_MAX = 0.85


def classify_blur(value: float) -> str:
    """
    Zwraca prostą flagę jakości ostrości.

    value – wartość metryki blur dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    return "ok" if value >= BLUR_MIN_GOOD else "too_blurry"


def classify_brightness(value: float) -> str:
    """
    Zwraca prostą flagę jakości jasności.

    value – wartość metryki brightness dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    if value < BRIGHTNESS_MIN:
        return "too_dark"
    if value > BRIGHTNESS_MAX:
        return "too_bright"
    return "ok"


def classify_contrast(value: float) -> str:
    """
    Zwraca prostą flagę jakości kontrastu.

    value – wartość metryki contrast dla pojedynczego obrazu
    """
    if pd.isna(value):
        return "unknown"
    if value < CONTRAST_MIN:
        return "too_low"
    if value > CONTRAST_MAX:
        return "too_high"
    return "ok"


df_quality["flag_blur"] = df_quality["blur"].map(classify_blur)
df_quality["flag_brightness"] = df_quality["brightness"].map(classify_brightness)
df_quality["flag_contrast"] = df_quality["contrast"].map(classify_contrast)


def combine_flags(row: pd.Series) -> str:
    """
    Łączy flagi cząstkowe w jedną flagę jakości.

    row – wiersz DataFrame z kolumnami flag_blur, flag_brightness, flag_contrast
    """
    flags = {row["flag_blur"], row["flag_brightness"], row["flag_contrast"]}
    if flags <= {"ok"}:
        return "ok"
    if "unknown" in flags and flags == {"unknown"}:
        return "unknown"
    return "bad"


df_quality["quality_flag"] = df_quality.apply(combine_flags, axis=1)

# Kolumna na notatki ludzkie – tylko jeśli jeszcze nie istnieje
if "quality_notes" not in df_quality.columns:
    df_quality["quality_notes"] = ""

In [11]:
### X. Podsumowanie globalne i po folderach

"""
Tworzy podsumowania metryk jakości:
- opis statystyczny dla całego zbioru,
- agregację po folderze nadrzędnym (np. serii skanów).

df_quality – DataFrame z metrykami i flagami jakości
"""

from pathlib import Path

# Kolumny pomocnicze: nazwa pliku i folder nadrzędny (jeśli jeszcze ich nie ma)
if "file_name" not in df_quality.columns:
    df_quality["file_name"] = df_quality["file_path"].map(lambda p: Path(p).name)

if "parent_folder" not in df_quality.columns:
    df_quality["parent_folder"] = df_quality["file_path"].map(lambda p: Path(p).parent.name)

summary_global = df_quality[["blur", "brightness", "contrast"]].describe(
    percentiles=[0.05, 0.5, 0.95]
)

summary_by_folder = (
    df_quality
    .groupby("parent_folder")[["blur", "brightness", "contrast"]]
    .agg(["count", "mean", "median"])
    .sort_values(("blur", "mean"))
)

print("=== Podsumowanie globalne ===")
print(summary_global)

print("\n=== Podsumowanie po folderach (posortowane po średnim blur) ===")
print(summary_by_folder.head(20))

=== Podsumowanie globalne ===
              blur  brightness   contrast
count    74.000000   74.000000  74.000000
mean    624.786369  129.621526  63.759943
std     711.216834   24.547015  10.305343
min      41.226857   80.469344  35.507076
5%       98.784922   92.655143  48.900228
50%     386.056257  126.265237  62.746498
95%    1798.335806  175.971043  83.490583
max    3991.757403  189.707936  95.203951

=== Podsumowanie po folderach (posortowane po średnim blur) ===
               blur                         brightness              \
              count        mean      median      count        mean   
parent_folder                                                        
inputs           74  624.786369  386.056257         74  129.621526   

                          contrast                        
                   median    count       mean     median  
parent_folder                                             
inputs         126.265237       74  63.759943  62.746498  


In [12]:
### X. Eksport list obrazów „do sprawdzenia”

"""
Eksportuje:
- globalny plik z metrykami jakości,
- listę najgorszych obrazów wg blur,
- listę obrazów oznaczonych jako „bad”.

df_quality     – DataFrame z metrykami jakości
OUTPUT_CSV_DIR – katalog wyjściowy na pliki CSV (zdefiniowany wcześniej w notatniku)
"""

from pathlib import Path

QUALITY_DIR = OUTPUT_CSV_DIR  # dostosuj, jeśli używasz innej zmiennej katalogu
QUALITY_DIR.mkdir(parents=True, exist_ok=True)

quality_all_path = QUALITY_DIR / "quality_metrics.csv"
worst_blur_path = QUALITY_DIR / "quality_worst_blur.csv"
bad_overall_path = QUALITY_DIR / "quality_bad_overall.csv"

# 1) Wszystkie metryki
df_quality.to_csv(quality_all_path, index=False)

# 2) Np. 10% najgorszych obrazów po blur (jeśli blur rośnie z ostrością, sortujemy rosnąco)
k = max(1, int(0.10 * len(df_quality)))
df_worst_blur = df_quality.sort_values("blur", ascending=True).head(k)
df_worst_blur.to_csv(worst_blur_path, index=False)

# 3) Wszystkie obrazy z łączną flagą jakości „bad”
df_bad = df_quality[df_quality["quality_flag"] == "bad"].copy()
df_bad.to_csv(bad_overall_path, index=False)

print("Zapisano:")
print(f"- wszystkie metryki: {quality_all_path}")
print(f"- ~10% najbardziej problematycznych (po blur): {worst_blur_path}")
print(f"- wszystkie obrazy z quality_flag == 'bad': {bad_overall_path}")

Zapisano:
- wszystkie metryki: /Users/olga/MetaLogic/outputs/csv/quality_metrics.csv
- ~10% najbardziej problematycznych (po blur): /Users/olga/MetaLogic/outputs/csv/quality_worst_blur.csv
- wszystkie obrazy z quality_flag == 'bad': /Users/olga/MetaLogic/outputs/csv/quality_bad_overall.csv
